In [ ]:
# This cell is auto-generated by Garden. Don't delete it. Do keep it as the first cell.
# It records the base image and requirements you passed to `garden-ai notebook start`.
# That way the next time you run this notebook Garden can start it with the same libraries.

"""
NOTEBOOK_METADATA = {
  "NOTEBOOK_BASE_IMAGE_NAME": "3.10-tensorflow",
  "NOTEBOOK_REQUIREMENTS": {
    "file_format": "pip",
    "contents": [
      "tensorflow==2.14.0"
    ]
  }
}
"""

In [2]:
from garden_ai.model_connectors import HFConnector
from garden_ai import EntrypointMetadata, garden_entrypoint, entrypoint_test

import numpy as np
from tensorflow.keras.models import load_model

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [3]:
my_hugging_face_repo = HFConnector("willengler-uc/keras-mnist")

### Pipeline metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this PipelineMetadata object to describe your function.


In [7]:
my_entrypoint_meta = EntrypointMetadata(
    title="Keras MNIST digit classifier",
    description="Classify fuzzy black and white digits with TensorFlow",
    authors=["Will Engler"],
    tags=["MNIST digits", "Keras"]
)

In [8]:
@garden_entrypoint(metadata=my_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.26311/b74a-5c58")
def classify_digit(image):
    download_path = my_hugging_face_repo.stage()
    model = load_model(f"{download_path}/mnist_model.keras")
    prediction = model.predict(image)
    return np.argmax(prediction)

In [9]:
@entrypoint_test(classify_digit)
def test_the_model():
    from tensorflow.keras.datasets import mnist

    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
    sample_image = test_images[0]
    sample_image = np.expand_dims(sample_image, axis=0)
    return classify_digit(sample_image)

In [10]:
test_the_model()